In [15]:
import os
import pandas as pd
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import logging


In [16]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

Celda 2: Carga de variables de entorno y definición de constantes


In [17]:
logging.info("Cargando variables de entorno...")
dotenv_path = '/home/nicolas/Escritorio/workshops/workshop_2/env/.env'
load_dotenv(dotenv_path=dotenv_path)

DB_USER = os.getenv('POSTGRES_USER')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD')
DB_HOST = os.getenv('POSTGRES_HOST')
DB_PORT = os.getenv('POSTGRES_PORT')
DB_NAME = os.getenv('POSTGRES_DB')
DEFAULT_DB = 'postgres' 

CSV_FILE_PATH = '/home/nicolas/Escritorio/workshops/workshop_2/data/the_grammy_awards.csv'
TABLE_NAME = 'the_grammy_awards'

logging.info(f"Variables cargadas: DB_HOST={DB_HOST}, DB_PORT={DB_PORT}, DB_NAME={DB_NAME}, DB_USER={DB_USER}")
if not all([DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME]):
    logging.error("Una o más variables de entorno no están definidas correctamente en " + dotenv_path)

2025-04-11 12:29:46,825 - INFO - Cargando variables de entorno...
2025-04-11 12:29:46,827 - INFO - Variables cargadas: DB_HOST=localhost, DB_PORT=5432, DB_NAME=artists, DB_USER=postgres


In [18]:
conn_default = None
cursor_default = None
try:
    logging.info(f"Intentando conectar a la base de datos por defecto '{DEFAULT_DB}' para verificar/crear '{DB_NAME}'...")
    conn_default = psycopg2.connect(
        dbname=DEFAULT_DB,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    conn_default.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
    cursor_default = conn_default.cursor()

    cursor_default.execute(sql.SQL("SELECT 1 FROM pg_database WHERE datname = %s"), (DB_NAME,))
    exists = cursor_default.fetchone()

    if not exists:
        logging.info(f"La base de datos '{DB_NAME}' no existe. Creándola...")
        cursor_default.execute(sql.SQL(f"CREATE DATABASE {DB_NAME}"))
        logging.info(f"Base de datos '{DB_NAME}' creada exitosamente.")
    else:
        logging.info(f"La base de datos '{DB_NAME}' ya existe.")

except psycopg2.Error as e:
    logging.error(f"Error al conectar o verificar/crear la base de datos '{DB_NAME}': {e}")
except Exception as e:
    logging.error(f"Ocurrió un error inesperado durante la verificación/creación de la base de datos: {e}")
finally:
    if cursor_default:
        cursor_default.close()
        logging.info("Cursor de la conexión por defecto cerrado.")
    if conn_default:
        conn_default.close()
        logging.info("Conexión a la base de datos por defecto cerrada.")

2025-04-11 12:29:46,834 - INFO - Intentando conectar a la base de datos por defecto 'postgres' para verificar/crear 'artists'...
DETAIL:  La base de datos fue creada usando la versión de ordenamiento 2.31, pero el sistema operativo provee la versión 2.35.
HINT:  Reconstruya todos los objetos en esta base de datos que usen el ordenamiento por omisión y ejecute ALTER DATABASE postgres REFRESH COLLATION VERSION, o construya PostgreSQL con la versión correcta de la biblioteca.
2025-04-11 12:29:46,844 - INFO - La base de datos 'artists' ya existe.
2025-04-11 12:29:46,846 - INFO - Cursor de la conexión por defecto cerrado.
2025-04-11 12:29:46,846 - INFO - Conexión a la base de datos por defecto cerrada.


Celda 4: Cargar datos desde el archivo CSV a un DataFrame de Pandas

In [19]:
                                                                     
logging.info(f"Cargando datos desde {CSV_FILE_PATH}...")
try:
    df = pd.read_csv(CSV_FILE_PATH)
    num_csv_rows = len(df)
    logging.info(f"Archivo CSV cargado exitosamente. Número de filas: {num_csv_rows}")
    logging.info("Primeras 5 filas del DataFrame:")
    print(df.head().to_markdown(index=False))                                                 
    logging.info("Información del DataFrame:")
    df.info()

except FileNotFoundError:
    logging.error(f"Error: El archivo CSV no se encontró en la ruta: {CSV_FILE_PATH}")
                                                  
    raise
except Exception as e:
    logging.error(f"Error al leer el archivo CSV: {e}")
    raise

2025-04-11 12:29:46,860 - INFO - Cargando datos desde /home/nicolas/Escritorio/workshops/workshop_2/data/the_grammy_awards.csv...
2025-04-11 12:29:46,879 - INFO - Archivo CSV cargado exitosamente. Número de filas: 4810
2025-04-11 12:29:46,880 - INFO - Primeras 5 filas del DataFrame:
2025-04-11 12:29:46,883 - INFO - Información del DataFrame:


|   year | title                             | published_at              | updated_at                | category           | nominee    | artist        | workers                                                                                                                                                                                       | img                                                                                                                                           | winner   |
|-------:|:----------------------------------|:--------------------------|:--------------------------|:-------------------|:-----------|:--------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------|:---------|


Celda 5: Conectar a la base de datos 'artists' y cargar el DataFrame


In [20]:
engine = None
try:
    logging.info(f"Creando motor SQLAlchemy para la base de datos '{DB_NAME}'...")
                                              
    db_url = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
    engine = create_engine(db_url)

    logging.info(f"Conectando a '{DB_NAME}' y cargando el DataFrame en la tabla '{TABLE_NAME}'...")
                                                         
                                                                          
                                                                 
    df.to_sql(TABLE_NAME, con=engine, if_exists='replace', index=False, method='multi')
    logging.info(f"DataFrame cargado exitosamente en la tabla '{TABLE_NAME}'.")

except Exception as e:
    logging.error(f"Error al cargar el DataFrame en la base de datos: {e}")
                                                    
    raise
                                                                                     
                                                                                    

2025-04-11 12:29:46,906 - INFO - Creando motor SQLAlchemy para la base de datos 'artists'...
2025-04-11 12:29:46,908 - INFO - Conectando a 'artists' y cargando el DataFrame en la tabla 'the_grammy_awards'...
2025-04-11 12:29:47,450 - INFO - DataFrame cargado exitosamente en la tabla 'the_grammy_awards'.


Celda 6: Verificar la carga de datos


In [21]:
if engine:
    try:
        logging.info(f"Verificando la carga en la tabla '{TABLE_NAME}'...")
        with engine.connect() as connection:
            query = text(f"SELECT COUNT(*) FROM {TABLE_NAME}")
            result = connection.execute(query)
            num_db_rows = result.scalar_one()

        logging.info(f"Número de filas en la tabla '{TABLE_NAME}': {num_db_rows}")

        if num_csv_rows == num_db_rows:
            logging.info("¡Verificación exitosa! El número de filas coincide entre el CSV y la base de datos.")
        else:
            logging.warning(f"Discrepancia en el número de filas: CSV ({num_csv_rows}) vs DB ({num_db_rows}).")

    except Exception as e:
        logging.error(f"Error durante la verificación de la carga: {e}")
else:
    logging.error("No se pudo realizar la verificación porque la conexión a la base de datos falló previamente.")

logging.info("Script finalizado.")

2025-04-11 12:29:47,456 - INFO - Verificando la carga en la tabla 'the_grammy_awards'...
2025-04-11 12:29:47,460 - INFO - Número de filas en la tabla 'the_grammy_awards': 4810
2025-04-11 12:29:47,460 - INFO - ¡Verificación exitosa! El número de filas coincide entre el CSV y la base de datos.
2025-04-11 12:29:47,461 - INFO - Script finalizado.
